##Import Necessary aaaaand Not so Necessary Libraries


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import roc_curve, auc, log_loss, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


##Data Preprocessing Stage


Basic EDA to understand:


*   Missing Data
*   Types of Data
*   Required Preprocessing



In [ ]:
data = pd.read_csv("weatherAUS.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

From the above we can tell that the dataset consists of several incomplete values that would require imputation.

Further, there are 16 float64 datatypes and 7 objects. So we make our imputing and encoding decisions based on that.

##Encoding Categorical Data
Making use of OneHotEncoding for different values for all categorical columns. The Dummy Variable count is taken care of by the model itself.


In [ ]:
cat_data=["Location","WindGustDir","WindDir9am","WindDir3pm","RainToday"]
encoder = OneHotEncoder()

for col in cat_data:
  # Transform the categorical data
  transformed_data = encoder.fit_transform(data[[col]])

  # Convert the sparse matrix to a dense array
  transformed_data = transformed_data.toarray()

  # Create a DataFrame from the transformed data
  df_transformed = pd.DataFrame(transformed_data, columns=encoder.get_feature_names_out([col]))
  data.drop(col,axis=1)
  # Concatenate the transformed data with the original DataFrame
  data = pd.concat([data, df_transformed], axis=1)

  # Remove the original categorical column
  data = data.drop(col, axis=1)

From our initial look at the dataset we can tell that in several but not too many rows the dependent variable itself is absent. We can delete the rows that have the missing dependent variable for simpler computation.

In [ ]:
data.dropna(subset=['RainTomorrow'], inplace=True)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 142193 entries, 0 to 145458
Columns: 121 entries, Date to RainToday_nan
dtypes: float64(119), object(2)
memory usage: 132.4+ MB
None


##Imputing values for Numerical Data

We use the mean for each column to enter our missing values.

In [ ]:
float_imputer = SimpleImputer(strategy='mean', missing_values=np.nan)

num_data=['MinTemp','MaxTemp','Rainfall','Evaporation','Sunshine','WindGustSpeed','WindSpeed9am','WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm','Cloud9am','Cloud3pm','Temp9am','Temp3pm']
for column in num_data:
  data[column]=float_imputer.fit_transform(data[[column]])


##Preparing Data

Since our data is now processed and complete, we can categorise them into dependent and independent variables (X and y)

In [ ]:

X = data.iloc[:,:-1]
X=X.drop(["Date","RainTomorrow"],axis=1)
print(X)
y = data.iloc[:,-1]
y=y.values

        MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0          13.4     22.9       0.6     5.469824  7.624853           44.0   
1           7.4     25.1       0.0     5.469824  7.624853           44.0   
2          12.9     25.7       0.0     5.469824  7.624853           46.0   
3           9.2     28.0       0.0     5.469824  7.624853           24.0   
4          17.5     32.3       1.0     5.469824  7.624853           41.0   
...         ...      ...       ...          ...       ...            ...   
145454      3.5     21.8       0.0     5.469824  7.624853           31.0   
145455      2.8     23.4       0.0     5.469824  7.624853           31.0   
145456      3.6     25.3       0.0     5.469824  7.624853           22.0   
145457      5.4     26.9       0.0     5.469824  7.624853           37.0   
145458      7.8     27.0       0.0     5.469824  7.624853           28.0   

        WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  ...  \
0               20

##Encoding the y variable

In [ ]:
label_encoder=LabelEncoder()
y=label_encoder.fit_transform(y)

##Split The Dataset Into Training And Testing Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

##Load and fit the model to our training data

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

##Evaluation and Testing

Using several methods/metrics that come with scikit-learn we may now evaluate our model



--- Basic Evaluation using test set



In [ ]:
y_pred = model.predict(X_test)
type(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9896972467386336
